# Training Korean Text


In [1]:
import time
import csv
import os
from pathlib import Path
import sys
sys.path.insert(0, '../')
import pycor.korlang as korlang
import pycor.utils as utils
from pycor.res import WikiResolver

stopwatch = utils.StopWatch()

docsize = 7000

#for aux in lm.auxmap.values():
#    print(aux)

def listfiles(path):
    result_arr = []
    filenames = os.listdir(path)
    for filename in filenames:
        full_filename = os.path.join(path, filename)
        result_arr.append(full_filename)
    return result_arr
    
files = listfiles('../samples')
files = sorted(files)
files.extend( listfiles('../../data/wiki2') )
files.extend( listfiles('../../data/wiki') )
files.extend( listfiles('../../data/NP') )

print (len(files), "files")

outputpath = "../../output/" + str(docsize) + "/"
model_path = outputpath + "model/"

os.makedirs(outputpath, exist_ok=True)
os.makedirs(model_path, exist_ok=True)


dictionary_path = "../samples/dic/dictionary.csv"

trainer = korlang._trainer

# 사전 등록 
trainer.loaddic(dictionary_path)
# wikiResolver = WikiResolver(trainer.wordmap)
# trainer.setresolver(wikiResolver)

Init Trainer
Init DocResolver
3239 files
Loading Dictionary from ../samples/dic/dictionary.csv
Load  ../samples/dic/dictionary.csv   소요시간: 0.003


In [2]:

stopwatch.start()

def extractHeads(words) :
    word_texts = []
    for word in words:
        if word.bestpair:
            word_texts.append(word.bestpair.head.text)
        else:
            word_texts.append(word.text)
    return word_texts


def extractTails(words) :
    word_texts = []
    for word in words:
        if word.bestpair:
            #word_texts.append(word.bestpair.tail.text + ":" + str(word.bestpair.type))
            word_texts.append(word.bestpair.tail.text)
        else:
            word_texts.append("")
            #word_texts.append(word.text)
    return word_texts



def extractTags(words) :
    word_texts = []
    for word in words:
        if word.bestpair:
            word_texts.append(word.bestpair.tags)
        else:
            word_texts.append("*")
    return word_texts



def writeTails(outputpath, index, sentence_array):
    outputfile = outputpath + 'samples/' + str(index) + '_tails.txt'
    
    words_array = trainer.scoreDocument(sentence_array)
    
    with open(outputfile, 'w', encoding='utf-8') as out:
        writer = csv.writer(out)
        for words in words_array:
            word_texts = extractTails(words)
            writer.writerow(word_texts)
        out.close()
        
def writeAll(tailswriter, headswriter, tagswriter, sentence_array):
    words_array = trainer.scoreDocument(sentence_array)
    
    for words in words_array:
        word_texts = extractTails(words)
        
        if len(word_texts) > 2:
            tailswriter.writerow(word_texts)
            heads = extractHeads(words)
            headswriter.writerow(heads)
            tags = extractTags(words)
            tagswriter.writerow(tags)



        
tailsfile = open(outputpath + 'tails.txt', 'w', encoding='utf-8')
tailswriter = csv.writer(tailsfile)


headsfile = open(outputpath + 'heads.txt', 'w', encoding='utf-8')
headswriter = csv.writer(headsfile)


tagsfile = open(outputpath + 'tags.txt', 'w', encoding='utf-8')
tagswriter = csv.writer(tagsfile)

print("Loading Texts >>> ")
index =0
for file in files[:docsize]:
    if file.endswith(".txt") :
        sentence_array, words_array = trainer.loadfile(file)
        writeAll(tailswriter,headswriter, tagswriter,sentence_array)
        index += 1
        if index % 100 == 0:
            print(index,end=">")
            if index % 1000 == 0:
                print()
            
tailsfile.close()
tagsfile.close()

print("Load Texts: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")



Loading Texts >>> 
100>200>300>400>500>600>700>800>900>1000>
1100>1200>1300>1400>1500>1600>1700>1800>1900>2000>
2100>2200>2300>2400>2500>2600>2700>2800>2900>3000>
3100>3200>Load Texts:  543s.990ms. ( 543,990 ms.)


In [3]:
def writeWord(writer, word):
    if len(word.particles) == 0:
        writer.writerow([word.text, 'X'])
    for part in word.particles:
        h = part.head.text if part.head else ''
        t = part.tail.text if part.tail else ''
        writer.writerow([word.text, h, t, part.score, part.tags, part.pos])

utils.writecsv(outputpath + "np_words.csv", trainer.wordmap.words.values(), writeWord)

def writeWord(writer, word):
    bestpair = word.bestpair
    if bestpair :
        h = bestpair.head.text if bestpair.head else ''
        t = bestpair.tail.text if bestpair.tail else 'X'
        writer.writerow([ word.text, '   ', h, t, '   ', bestpair.score, bestpair.tags, bestpair.pos ])
    else:
        writer.writerow([ word.text, 'X' ])
        
utils.writecsv(outputpath + "np_words_scored.csv", trainer.wordmap.words.values(), writeWord)

Write  ../../output/7000/np_words.csv   소요시간: 6s.055ms. ( 6,054 ms.)
Write  ../../output/7000/np_words_scored.csv   소요시간: 3s.507ms. ( 3,507 ms.)


In [4]:

stopwatch.start()

print("Build Vocab: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")
print("Word Count : " , utils.comma(len(trainer.wordmap.words)) )
print("head Count : " , utils.comma(len(trainer.wordmap.heads)) )
print("tail Count : " , utils.comma(len(trainer.wordmap.tails)) )



snglist, ylist, clist, ambilist = trainer.buildVocab()


trainer.savemodel(model_path)

# with open(model_path + "ambiguous.csv", 'w', encoding='utf-8') as csvfile :
#     writer = csv.writer(csvfile)
#     for head in ambilist:
#         writer.writerow([head.text, '+'.join(head.pos), head.tails])
            
#     csvfile.close()

# with open(model_path + "single.csv", 'w', encoding='utf-8') as csvfile :
#     writer = csv.writer(csvfile)
#     for head in snglist:
#         writer.writerow([head.text, '+'.join(head.pos), head.tails])
            
#     csvfile.close()

Build Vocab:  0s.000ms. ( 0 ms.)
Word Count :  660,119
head Count :  470,794
tail Count :  1,182
Single Count: 204676
용언 Count: 25636
체언 Count: 105617
Ambiguous Count: 11936
Heads Count: 470794
Tails Count: 1182
Clear words size: 660119
Saving Model to ../../output/7000/model/
Save  ../../output/7000/model//heads.csv   소요시간: 1.705
Save  ../../output/7000/model//tails.csv   소요시간: 0.01
Clear Heads: 321200 remains.


In [5]:
def writeTails(writer, tail):
    #row = [tail.text]
    #for head in tail.heads:
    #    row.append(head.text)
    #row = [tail.text, len(tail.heads), tail.heads]
    row = [tail.text, len(tail.heads)]
    writer.writerow(row)

tailsorted = sorted(trainer.wordmap.tails.values(), key=lambda tail:tail.text[len(tail.text)-1])
utils.writecsv(outputpath + "np_tails.csv", tailsorted, writeTails)


tailsorted2 = sorted(trainer.wordmap.tails.values(), key=lambda tail:len(tail.heads), reverse=True)
utils.writecsv(outputpath + "np_tails_occ.csv", tailsorted2, writeTails)



Write  ../../output/7000/np_tails.csv   소요시간: 0s.008ms. ( 7 ms.)
Write  ../../output/7000/np_tails_occ.csv   소요시간: 0s.010ms. ( 9 ms.)


In [6]:
def writeHeads(writer, head):
    row = [ str(head), '  ', len(head.tails), ' ']
    for tail in head.tails:
        row.append(tail.text)
    writer.writerow(row)

headsorted = sorted(trainer.wordmap.heads.values(), key=lambda head:head.text)
utils.writecsv(outputpath + "np_heads.csv", headsorted, writeHeads)


headsorted2 = sorted(trainer.wordmap.heads.values(), key=lambda head:len(head.tails), reverse=True)
utils.writecsv(outputpath + "np_heads_occ.csv", headsorted2, writeHeads)


headset = set()

for word in trainer.wordmap.words.values():
    bestpair = word.bestpair
    if bestpair and bestpair.head:
        headset.add(bestpair.head)
        
def writeHead(writer, head):
    row = [ str(head), '\t', len(head.tails), '\t']
    
    for tail in head.tails:
        row.append(tail.text)
        
    writer.writerow(row)
        

        
headlist = sorted(headset , key=lambda head:head.text )

utils.writecsv(outputpath + "np_head_scored.csv", headlist, writeHead)




Write  ../../output/7000/np_heads.csv   소요시간: 0s.862ms. ( 861 ms.)
Write  ../../output/7000/np_heads_occ.csv   소요시간: 0s.804ms. ( 803 ms.)
Write  ../../output/7000/np_head_scored.csv   소요시간: 1s.144ms. ( 1,143 ms.)


In [7]:

def extractWords(words) :
    word_texts = []
    for word in words:
        if word.bestpair:
            #word_texts.append(word.bestpair.head.text +":"+word.bestpair.tail.text)
            pos = word.bestpair.pos if word.bestpair.pos else ''
            tag = word.bestpair.tags if word.bestpair.tags else ''
#             print(word.bestpair.tail.occurence())
            postag = None
            if pos or tag:
                postag = "("+ str(pos) + ":" + str(tag) +")"
            else :
                postag = ""
            word_texts.append(word.bestpair.head.text + postag)
        else:
            word_texts.append(word.text)
    return word_texts

def writeExtracted(outputpath, suffix, index, sentence_array):
    outputfile = outputpath + str(index) + suffix + '.txt'
    
    words_array = trainer.scoreDocument(sentence_array)

    with open(outputfile, 'w', encoding='utf-8') as out:
        writer = csv.writer(out)
        for words in words_array:
            word_texts = extractWords(words)
            writer.writerow(word_texts)
        out.close()
        
index = 0
for file in files[:5]:
    if file.endswith(".txt") :
        sentence_array, words_array = trainer.loadfile(file)
        index += 1
        writeExtracted(outputpath, '_train', index, sentence_array)

import pycor
pycor.loadmodel(model_path)

index = 0
for file in files[:5]:
    if file.endswith(".txt") :
        sentence_array, words_array = pycor.readfile(file)
        index += 1
        writeExtracted(outputpath, '_pycor', index, sentence_array)




Loading Model from ../../output/7000/model/
Load  ../../output/7000/model//heads.csv   소요시간: 1.086
Load  ../../output/7000/model//tails.csv   소요시간: 0.013
